In [1]:
pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j langchain-groq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have goog

In [ ]:
from langchain_community.graphs import Neo4jGraph
import os
os.environ["NEO4J_URI"] = "neo4j+s://80e9be90.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "..."

graph = Neo4jGraph(refresh_schema=False)

In [3]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = "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"

 ## Generation of MAIN Acedamic Knowledge Graph

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_groq import ChatGroq
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)

In [21]:
from langchain_core.documents import Document

with open("/kaggle/input/dl-project-dataset-interview/Knowledge INPUT.txt", "r") as file:
    text = file.read()

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mathematics', type='Field', properties={}), Node(id='Linear Algebra', type='Subset', properties={}), Node(id='Calculus', type='Subset', properties={}), Node(id='Probability', type='Subset', properties={}), Node(id='Statistics', type='Subset', properties={}), Node(id='Discrete Mathematics', type='Subset', properties={}), Node(id='Graph Theory', type='Subset', properties={}), Node(id='Number Theory', type='Subset', properties={}), Node(id='Combinatorics', type='Subset', properties={}), Node(id='Programming Languages', type='Tools', properties={}), Node(id='Computer Science', type='Field', properties={}), Node(id='Object-Oriented Programming', type='Paradigm', properties={}), Node(id='Functional Programming', type='Paradigm', properties={}), Node(id='Procedural Programming', type='Paradigm', properties={}), Node(id='Java', type='Language', properties={}), Node(id='Python', type='Language', properties={}), Node(id='C++', type='Language', properties={}), Node(id='Javascript'

In [22]:
graph.add_graph_documents(graph_documents)
print("Graph data has been stored in Neo4j.")

Graph data has been stored in Neo4j.


## Entity extraction

In [5]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from typing import Dict, List

# Instantiate the ChatGroq model
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.1,
)

def create_system_prompt() -> str:
    return """You are a skill extraction specialist. Extract ONLY specific named skills, tools, technologies, and knowledge areas. 
    - Extract single-word or short-phrase skills ONLY
    - DO NOT include full sentences or descriptions
    - DO NOT include experience levels or qualifications
    - Each skill should be a clear, standalone term"""

def create_skill_extraction_prompt(text: str) -> str:
    return f"""Extract specific skills from the text below into these categories:

        Technical Skills: Specific tools, programming languages, software, platforms
        Example: Python, SQL, Tableau, Excel

        Domain Knowledge: Specific fields, subjects, methodologies
        Example: Machine Learning, Statistics, Data Mining


        Text: {text}

        Return ONLY in this JSON format:
        {{
            "technical_skills": ["skill1", "skill2"],
            "domain_knowledge": ["domain1", "domain2"],
        }}"""

def extract_skills(text: str) -> Dict[str, List[str]]:
    messages = [
        ("system", create_system_prompt()),
        ("human", create_skill_extraction_prompt(text))
    ]
    
    response = llm.invoke(messages)
    json_parser = JsonOutputParser()
    
    try:
        parsed_skills = json_parser.parse(response.content)
        return parsed_skills
    except Exception as e:
        print(f"Error parsing response: {e}")
        return {
            "technical_skills": [],
            "domain_knowledge": [],
        }

### Job Description Skills Extraction

In [6]:
# Job Description
text = """Sample data analyst job description
At [Company X], we’re proud to stand at the forefront of the Big Data revolution. Using the latest analytics tools and processes, we’re able to maximize our offerings and deliver unparalleled service and support. To help carry us even further, we’re searching for an experienced data analyst to join our team. The ideal candidate will be highly skilled in all aspects of data analytics, including mining, generation, and visualization. Additionally, this person should be committed to transforming data into readable, goal-oriented reports that drive innovation and growth.

Objectives of this role
Develop, implement, and maintain leading-edge analytics systems, taking complicated problems and building simple frameworks
Identify trends and opportunities for growth through analysis of complex datasets
Evaluate organizational methods and provide source-to-target mappings and information-model specification documents for datasets
Create best-practice reports based on data mining, analysis, and visualization
Evaluate internal systems for efficiency, problems, and inaccuracies, and develop and maintain protocols for handling, processing, and cleaning data
Work directly with managers and users to gather requirements, provide status updates, and build relationships
Responsibilities
Work closely with project managers to understand and maintain focus on their analytics needs, including critical metrics and KPIs, and deliver actionable insights to relevant decision-makers
Proactively analyze data to answer key questions for stakeholders or yourself, with an eye on what drives business performance, and investigate and communicate which areas need improvement in efficiency and productivity
Create and maintain rich interactive visualizations through data interpretation and analysis, with reporting components from multiple data sources
Define and implement data acquisition and integration logic, selecting an appropriate combination of methods and tools within the defined technology stack to ensure optimal scalability and performance of the solution
Develop and maintain databases by acquiring data from primary and secondary sources, and build scripts that will make our data evaluation process more flexible or scalable across datasets
Required skills and qualifications
Three or more years of experience mining data as a data analyst
Proven analytics skills, including mining, evaluation, and visualization
Technical writing experience in relevant areas, including queries, reports, and presentations
Strong SQL or Excel skills, with aptitude for learning other analytics tools
Preferred skills and qualifications
Bachelor’s degree (or equivalent) in mathematics, computer science, economics, or statistics
Experience with database and model design and segmentation techniques
Strong programming experience with frameworks, including XML, JavaScript, and ETL
Practical experience in statistical analysis through the use of statistical packages, including Excel, SPSS, and SAS
Proven success in a collaborative, team-oriented environment
"""

JD_skills = extract_skills(text)
print(JD_skills)

{'technical_skills': ['SQL', 'Excel', 'XML', 'JavaScript', 'ETL', 'SPSS', 'SAS'], 'domain_knowledge': ['Data Mining', 'Data Analytics', 'Data Visualization', 'Statistics', 'Mathematics', 'Computer Science', 'Economics']}


In [7]:
def create_relationship_prompt(new_skills: Dict[str, List[str]], existing_knowledge: Dict[str, List[str]]) -> str:
    return f"""Given:

EXISTING KNOWLEDGE GRAPH:
Technical Skills: {', '.join(existing_knowledge)}


NEW SKILLS TO INTEGRATE:
Technical Skills: {', '.join(new_skills['technical_skills'])}
Domain Knowledge: {', '.join(new_skills['domain_knowledge'])}

Generate ONLY accurate and well-established relationships between skills following these rules:
1. Only include relationships that are widely accepted in academia/industry
2. Focus on direct, clear hierarchical relationships
3. For tools, only state their primary purpose
4. Avoid broad or questionable generalizations
5. Each skill should only be related to its most direct parent/application

Return relationships in numbered list format.
Only include relationships you are completely confident about.Do not include any other text, explanations, or notes."""

In [44]:
from langchain_groq import ChatGroq
from typing import Dict, List

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.9,
)

existing_knowledge = node_names

new_skills = JD_skills
relationships = llm.invoke([
    ("system", "You are a knowledge graph relationship specialist. Create clear, ONLY really close relationships between technical concepts."),
    ("human", create_relationship_prompt(new_skills, existing_knowledge))
])

meaning=relationships.content


llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)
text = meaning
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

graph = Neo4jGraph(refresh_schema=False)
graph.add_graph_documents(graph_documents)

Nodes:[Node(id='Spss', type='Software', properties={}), Node(id='Sas', type='Software', properties={}), Node(id='Excel', type='Software', properties={}), Node(id='Xml', type='Language', properties={}), Node(id='Sql', type='Language', properties={}), Node(id='Etl', type='Process', properties={}), Node(id='Statistics', type='Field', properties={}), Node(id='Data Visualization', type='Field', properties={}), Node(id='Database Systems', type='Field', properties={})]
Relationships:[Relationship(source=Node(id='Spss', type='Software', properties={}), target=Node(id='Statistics', type='Field', properties={}), type='USED_FOR', properties={}), Relationship(source=Node(id='Sas', type='Software', properties={}), target=Node(id='Statistics', type='Field', properties={}), type='USED_FOR', properties={}), Relationship(source=Node(id='Excel', type='Software', properties={}), target=Node(id='Data Visualization', type='Field', properties={}), type='USED_FOR', properties={}), Relationship(source=Node(id

### Resume Skills Extraction

In [8]:
!pip install PyMuPDF python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.4 MB/s eta 0:00:00


#### Resume parser

In [9]:
import fitz
import docx
from pathlib import Path
from typing import Union

class ResumeParser:
    def __init__(self, file_path: Union[str, Path]):
        self.file_path = Path(file_path)
        self.text = ""

    def extract_text(self) -> str:
        """Extract text based on file extension"""
        suffix = self.file_path.suffix.lower()
        
        try:
            if suffix == '.pdf':
                return self._extract_from_pdf()
            elif suffix == '.docx':
                return self._extract_from_docx()
            elif suffix == '.txt':
                return self._extract_from_txt()
            else:
                raise ValueError(f"Unsupported file format: {suffix}")
        except Exception as e:
            print(f"Error extracting text from {self.file_path}: {str(e)}")
            return ""

    def _extract_from_pdf(self) -> str:
        """Extract text from PDF using PyMuPDF"""
        text = []
        with fitz.open(self.file_path) as doc:
            for page in doc:
                text.append(page.get_text())
        return "\n".join(text)

    def _extract_from_docx(self) -> str:
        """Extract text from DOCX"""
        doc = docx.Document(self.file_path)
        return "\n".join([paragraph.text for paragraph in doc.paragraphs])

    def _extract_from_txt(self) -> str:
        """Extract text from TXT"""
        return self.file_path.read_text(encoding='utf-8')

In [11]:
def extract_resume_text(file_path: str) -> str:
    parser = ResumeParser(file_path)
    return parser.extract_text()

    
resume_file = "/kaggle/input/resume/resume_v2.3.pdf"
text = extract_resume_text(resume_file)
print(text)

Revanth Srinivasa Reddy
Hyderabad, India 500090
 +91 9492891337
# revanthbommareddy@gmail.com
ï LinkedIn
§ GitHub
Education
Vellore Institute of Technology
Sep. 2022 – Present
Bachelor of Technology in Computer Science with specialization in AI & ML, CGPA: 9.22/10
Chennai, Tamil Nadu
Sri Chaitanya Junior College
April. 2020 – June. 2022
State Board (12th grade), Percentage: 92.9%
Hyderabad, Telangana
Vikas The Concept School
Mar. 2019 – Mar. 2020
CBSE (10th grade) , Percentage: 94.6%
Hyderabad, Telangana
Relevant Coursework
• Data Structures and
Algorithms
• Generative AI
• Natural Language
Processing
• Machine Learning
• Computer Vision
Experience
Samsung R&D Institute, Bengaluru
July 2024 – Present
Samsung PRISM Intern
Remote
• Researching integrating Large Language Models (LLMs) into recommendation systems to demonstrate their superiority
over existing methods by aiming for a performance improvement of at least 20%.
• Investigating methods to improve item titles and personalization

In [12]:
resume_skills = extract_skills(text)
print(resume_skills)

{'technical_skills': ['Python', 'Java', 'C', 'C++', 'HTML/CSS', 'JavaScript', 'TypeScript', 'SQL', 'R', 'Pandas', 'Numpy', 'Matplotlib', 'Seaborn', 'scikit-learn', 'Tensorflow', 'Keras', 'PyTorch', 'OpenCV', 'Streamlit', 'Langchain', 'ReactJS', 'NextJS', 'FastAPI', 'Flask', 'AWS', 'Microsoft Azure', 'Linux'], 'domain_knowledge': ['Machine Learning', 'Generative AI', 'Natural Language Processing', 'Computer Vision', 'Data Structures and Algorithms', 'Statistics', 'Data Mining']}


## Input Text creation for Knowledge Graph generation

In [19]:
def get_all_node_names():
    query = "MATCH (n) RETURN n"
    nodes = graph.query(query)
    for node in nodes:
        print(node)
    return [node['n']['id'] for node in nodes]

node_names = get_all_node_names()
print("Node Names:")
for name in node_names:
    print(name)


{'n': {'id': 'Mathematics'}}
{'n': {'id': 'Linear Algebra'}}
{'n': {'id': 'Calculus'}}
{'n': {'id': 'Probability'}}
{'n': {'id': 'Statistics'}}
{'n': {'id': 'Discrete Mathematics'}}
{'n': {'id': 'Graph Theory'}}
{'n': {'id': 'Number Theory'}}
{'n': {'id': 'Combinatorics'}}
{'n': {'id': 'Programming Languages'}}
{'n': {'id': 'Computer Science'}}
{'n': {'id': 'Object-Oriented Programming'}}
{'n': {'id': 'Functional Programming'}}
{'n': {'id': 'Procedural Programming'}}
{'n': {'id': 'Java'}}
{'n': {'id': 'Python'}}
{'n': {'id': 'C++'}}
{'n': {'id': 'Javascript'}}
{'n': {'id': 'Assembly Language'}}
{'n': {'id': 'Database Systems'}}
{'n': {'id': 'Sql'}}
{'n': {'id': 'Nosql'}}
{'n': {'id': 'Mongodb'}}
{'n': {'id': 'Postgresql'}}
{'n': {'id': 'Database Design'}}
{'n': {'id': 'Data Modeling'}}
{'n': {'id': 'Normalization'}}
{'n': {'id': 'Query Optimization'}}
{'n': {'id': 'Artificial Intelligence'}}
{'n': {'id': 'Machine Learning'}}
{'n': {'id': 'Deep Learning'}}
{'n': {'id': 'Neural Networks'

In [20]:
from langchain_groq import ChatGroq
from typing import Dict, List

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.9,
)

# def create_relationship_prompt(new_skills: Dict[str, List[str]], existing_knowledge: Dict[str, List[str]]) -> str:
#     return f"""Given:

# EXISTING KNOWLEDGE GRAPH:
# Technical Skills: {', '.join(existing_knowledge)}


# NEW SKILLS TO INTEGRATE:
# Technical Skills: {', '.join(new_skills['technical_skills'])}
# Domain Knowledge: {', '.join(new_skills['domain_knowledge'])}

# Generate ONLY accurate and well-established relationships between skills following these rules:
# 1. Only include relationships that are widely accepted in academia/industry
# 2. Focus on direct, clear hierarchical relationships
# 3. For tools, only state their primary purpose
# 4. Avoid broad or questionable generalizations
# 5. Each skill should only be related to its most direct parent/application

# Return relationships in numbered list format.
# Only include relationships you are completely confident about.Do not include any other text, explanations, or notes."""

existing_knowledge = node_names

new_skills = resume_skills
relationships = llm.invoke([
    ("system", "You are a knowledge graph relationship specialist. Create clear, ONLY really close relationships between technical concepts."),
    ("human", create_relationship_prompt(new_skills, existing_knowledge))
])

meaning=relationships.content

In [21]:
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0.9)
llm_transformer = LLMGraphTransformer(llm=llm)
text = meaning
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mathematics', type='Field of study', properties={}), Node(id='Linear Algebra', type='Field of study', properties={}), Node(id='Calculus', type='Field of study', properties={}), Node(id='Probability', type='Field of study', properties={}), Node(id='Statistics', type='Field of study', properties={}), Node(id='Data Science', type='Field of study', properties={}), Node(id='Predictive Analytics', type='Field of study', properties={}), Node(id='Feature Engineering', type='Field of study', properties={}), Node(id='Data Mining', type='Field of study', properties={}), Node(id='Big Data', type='Field of study', properties={}), Node(id='Data Visualization', type='Field of study', properties={}), Node(id='Discrete Mathematics', type='Field of study', properties={}), Node(id='Graph Theory', type='Field of study', properties={}), Node(id='Combinatorics', type='Field of study', properties={}), Node(id='Number Theory', type='Field of study', properties={}), Node(id='Computer Science', 

In [23]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(refresh_schema=False)
graph.add_graph_documents(graph_documents)

In [55]:
JD_skills

{'technical_skills': ['SQL',
  'Excel',
  'XML',
  'JavaScript',
  'ETL',
  'SPSS',
  'SAS'],
 'domain_knowledge': ['Data Mining',
  'Data Analytics',
  'Data Visualization',
  'Statistics',
  'Machine Learning',
  'Database Design',
  'Statistical Analysis']}

In [16]:
resume_skills

{'technical_skills': ['Python',
  'Java',
  'C',
  'C++',
  'HTML/CSS',
  'JavaScript',
  'TypeScript',
  'SQL',
  'R',
  'Pandas',
  'Numpy',
  'Matplotlib',
  'Seaborn',
  'scikit-learn',
  'Tensorflow',
  'Keras',
  'PyTorch',
  'OpenCV',
  'Streamlit',
  'Langchain',
  'ReactJS',
  'NextJS',
  'FastAPI',
  'Flask',
  'AWS',
  'Microsoft Azure',
  'Linux'],
 'domain_knowledge': ['Machine Learning',
  'Generative AI',
  'Natural Language Processing',
  'Computer Vision',
  'Data Structures and Algorithms',
  'Statistics',
  'Data Mining']}

In [53]:
jd_skil = [skill for skills in JD_skills.values() for skill in skills]
resume_skil = [skill for skills in resume_skills.values() for skill in skills]


In [54]:
resume_skil

['Python',
 'PyTorch',
 'LangChain',
 'scikit-learn',
 'HTML',
 'CSS',
 'Javascript',
 'ReactJS',
 'FastAPI',
 'Pandas',
 'Numpy',
 'Keras',
 'C',
 'JAVA',
 'C++',
 'Oracle SQL',
 'Machine Learning',
 'Generative AI',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Neural networks',
 'Clustering algorithms',
 'Model aggregation',
 'Template extraction',
 'Feature matching',
 'Fraud detection',
 'OCR',
 'ANN Model',
 'Cardiovascular Disorders']

In [ ]:
from typing import List, Dict, Tuple

def calculate_skill_match_score(jd_skills: List[str], resume_skills: List[str], graph) -> Tuple[float, Dict]:
    """
    Calculate the skill match score between a job description and resume.
    Returns both the final score and detailed metrics.
    
    The scoring considers:
    1. Path distances between matched skills
    2. Coverage ratio (how many JD skills were matched)
    3. Relevance ratio (quality of the matches based on distance)
    """
    print("Testing JD skills...")
    jd_skill_ids = get_skill_ids(jd_skills, graph)
    print("\nTesting Resume skills...")
    resume_skill_ids = get_skill_ids(resume_skills, graph)

    query = """
    UNWIND $jd_skill_ids AS jd_skill_id
    UNWIND $resume_skill_ids AS resume_skill_id
    MATCH (jd_skill) WHERE id(jd_skill) = jd_skill_id
    MATCH (resume_skill) WHERE id(resume_skill) = resume_skill_id
    MATCH path = shortestPath((resume_skill)-[*..10]-(jd_skill))
    WHERE id(jd_skill) <> id(resume_skill)  // Exclude self-referential paths
    RETURN 
        id(jd_skill) AS jd_skill,
        id(resume_skill) AS resume_skill,
        length(path) AS distance
    """

    params = {'jd_skill_ids': jd_skill_ids, 'resume_skill_ids': resume_skill_ids}
    try:
        results = graph.query(query, params)
    except Exception as e:
        print(f"Debug: Query execution failed with error: {e}")
        return 0.0, {"error": str(e)}

    total_distance_score = 0
    matched_jd_skills = set()
    distance_scores = []  
    
    for result in results:
        distance = result.get('distance')
        if distance is None:
            continue
        
        distance_score = 1.0 / (distance + 1)
        total_distance_score += distance_score
        distance_scores.append(distance_score)
        
        matched_jd_skills.add(result['jd_skill'])

    total_jd_skills = len(jd_skill_ids)
    matched_skills_count = len(matched_jd_skills)
    coverage_ratio = matched_skills_count / total_jd_skills if total_jd_skills > 0 else 0

    avg_distance_score = (sum(distance_scores) / len(distance_scores)) if distance_scores else 0
    
    coverage_weight = 0.6
    relevance_weight = 1 - coverage_weight
    
    final_score = (coverage_weight * coverage_ratio) + (relevance_weight * avg_distance_score)

    metrics = {
        "final_score": final_score,
        "coverage_ratio": coverage_ratio,
        "avg_distance_score": avg_distance_score,
        "matched_skills_count": matched_skills_count,
        "total_jd_skills": total_jd_skills,
        "unmatched_skills_count": total_jd_skills - matched_skills_count
    }

    print(f"\nDetailed Matching Metrics:")
    print(f"Coverage Ratio: {coverage_ratio:.2f}")
    print(f"Average Distance Score: {avg_distance_score:.2f}")
    print(f"Matched Skills: {matched_skills_count}/{total_jd_skills}")
    print(f"Final Score: {final_score:.2f}")

    return final_score, metrics

def test_single_user_and_jd(graph, jd_skills: Dict[str, List[str]], resume_skills: Dict[str, List[str]]):
    """
    Test matching with sample skills and print detailed metrics.
    """
    jd_skills_flat = normalize_skills([skill for skills in jd_skills.values() for skill in skills])
    resume_skills_flat = normalize_skills([skill for skills in resume_skills.values() for skill in skills])
    
    match_score, metrics = calculate_skill_match_score(jd_skills_flat, resume_skills_flat, graph)
    
    print("\nMatch Analysis Summary:")
    print(f"Final Match Score: {match_score:.2f}")
    print(f"Coverage of JD Skills: {metrics['coverage_ratio']*100:.1f}%")
    print(f"Average Quality of Matches: {metrics['avg_distance_score']:.2f}")
    print(f"Unmatched JD Skills: {metrics['unmatched_skills_count']}")

def normalize_skills(skills: List[str]) -> List[str]:
    return [skill.lower() for skill in skills]

def get_skill_ids(skills: List[str], graph) -> List[int]:
    skill_ids = []
    query = """
    MATCH (n) 
    WHERE toLower(n.id) = $skill_name
    RETURN id(n) AS node_id, n.id as found_name
    """
    
    for skill in skills:
        try:
            result = graph.query(query, {'skill_name': skill})
            if result:
                skill_ids.append(result[0]['node_id'])
                print(f"Found skill: {result[0]['found_name']}")
            else:
                print(f"No match found for: {skill}")
        except Exception as e:
            print(f"Query error for {skill}: {e}")
    
    return skill_ids


test_single_user_and_jd(graph, JD_skills, resume_skills)

Testing JD skills...
Found skill: Sql
Found skill: Excel
Found skill: Xml
Found skill: Javascript
Found skill: Etl
Found skill: Spss
Found skill: Sas
Found skill: Data Mining
No match found for: data analytics
Found skill: Data Visualization
Found skill: Statistics
Found skill: Mathematics
Found skill: Computer Science
No match found for: economics

Testing Resume skills...
Found skill: Python
Found skill: Java
Found skill: C
Found skill: C++
Found skill: Html/Css
Found skill: Javascript
Found skill: Typescript
Found skill: Sql
Found skill: R
Found skill: Pandas
Found skill: Numpy
Found skill: Matplotlib
Found skill: Seaborn
Found skill: Scikit-Learn
Found skill: Tensorflow
Found skill: Keras
Found skill: Pytorch
Found skill: Opencv
Found skill: Streamlit
Found skill: Langchain
Found skill: Reactjs
Found skill: Nextjs
Found skill: Fastapi
No match found for: flask
Found skill: Aws
Found skill: Microsoft Azure
Found skill: Linux
Found skill: Machine Learning
Found skill: Generative Ai
F

# Multipe Users

In [ ]:
import os
from pathlib import Path
import pandas as pd

results_df = pd.DataFrame(columns=['Resume_Name', 'Match_Score', 'Skills_Score', 'Graph_Score'])

resume_folder = Path("/kaggle/input/resume/")

resume_data = []
for resume_file in resume_folder.glob("*.pdf"):
    try:
        resume_text = extract_resume_text(str(resume_file))
        resume_skills = extract_skills(resume_text)
        
        node_names = get_all_node_names()
        
        llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0.9)
        relationships = llm.invoke([
            ("system", "You are a knowledge graph relationship specialist. Create clear, ONLY really close relationships between technical concepts."),
            ("human", create_relationship_prompt(resume_skills, existing_knowledge=node_names))
        ])
        
        # add to graph
        text = relationships.content
        documents = [Document(page_content=text)]
        graph_documents = llm_transformer.convert_to_graph_documents(documents)
        graph.add_graph_documents(graph_documents)
        
        resume_data.append({
            'file_name': resume_file.name,
            'skills': resume_skills
        })
        
    except Exception as e:
        print(f"Error processing {resume_file.name}: {str(e)}")

In [31]:
def test_single_user_and_jd(graph, jd_skills: Dict[str, List[str]], resume_skills: Dict[str, List[str]]):
    """
    Test matching with sample skills and return detailed metrics.
    """
    # Flatten and normalize skills
    jd_skills_flat = normalize_skills([skill for skills in jd_skills.values() for skill in skills])
    resume_skills_flat = normalize_skills([skill for skills in resume_skills.values() for skill in skills])
    
    match_score, metrics = calculate_skill_match_score(jd_skills_flat, resume_skills_flat, graph)
    
    print("\nMatch Analysis Summary:")
    print(f"Final Match Score: {match_score:.2f}")
    print(f"Coverage of JD Skills: {metrics['coverage_ratio']*100:.1f}%")
    print(f"Average Quality of Matches: {metrics['avg_distance_score']:.2f}")
    print(f"Unmatched JD Skills: {metrics['unmatched_skills_count']}")
    
    return (
        match_score,
        metrics['coverage_ratio'],
        metrics['avg_distance_score']
    )

print("\nCalculating scores...")
for resume in resume_data:
    try:
        scores = test_single_user_and_jd(graph, JD_skills, resume['skills'])
        match_score, skills_score, graph_score = scores
        
        results_df.loc[len(results_df)] = {
            'Resume_Name': resume['file_name'],
            'Match_Score': match_score,
            'Skills_Score': skills_score,
            'Graph_Score': graph_score
        }
        print(f"Scored {resume['file_name']}: Match={match_score:.2f}, Skills={skills_score:.2f}, Graph={graph_score:.2f}")
        
    except Exception as e:
        print(f"Error in scoring {resume['file_name']}: {str(e)}")

if not results_df.empty:
    results_df_sorted = results_df.sort_values(by='Match_Score', ascending=False)
    print("\nRanked Resumes:")
    print(results_df_sorted)
    results_df_sorted.to_csv('resume_rankings.csv', index=False)
else:
    print("\nNo results to display - check the errors above")


Calculating scores...
Testing JD skills...
Found skill: Sql
Found skill: Excel
Found skill: Xml
Found skill: Javascript
Found skill: Etl
Found skill: Spss
Found skill: Sas
Found skill: Data Mining
No match found for: data analytics
Found skill: Data Visualization
Found skill: Statistics
Found skill: Mathematics
Found skill: Computer Science
No match found for: economics

Testing Resume skills...
Found skill: Python
Found skill: Html
Found skill: Css
Found skill: Sql
Found skill: Postgresql
Found skill: Oracle
Found skill: Javascript
Found skill: Angular
Found skill: Django
Found skill: Rest Apis
Found skill: Graphql
Found skill: Aws
Found skill: Redshift
Found skill: S3
Found skill: Git
Found skill: Selenium
Found skill: D3.Js

Detailed Matching Metrics:
Coverage Ratio: 0.83
Average Distance Score: 0.18
Matched Skills: 10/12
Final Score: 0.57

Match Analysis Summary:
Final Match Score: 0.57
Coverage of JD Skills: 83.3%
Average Quality of Matches: 0.18
Unmatched JD Skills: 2
Scored pyt

# Full FLow

In [ ]:
# 1. Create Main Knowledge Graph
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_groq import ChatGroq
from langchain_core.documents import Document

# Setup Neo4j connection
os.environ["NEO4J_URI"] = "neo4j+s://80e9be90.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j" 
os.environ["NEO4J_PASSWORD"] = "P2jyV6JZdfGh1K4MsVDwrWz4GuboXH7Uc4JKPaElzfE"
os.environ["GROQ_API_KEY"] = "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"

graph = Neo4jGraph(refresh_schema=False)
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)

# Load and transform knowledge base
with open("/kaggle/input/dl-project-dataset-interview/Knowledge INPUT.txt", "r") as file:
    text = file.read()
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
#graph.add_graph_documents(graph_documents)

# 2. Extract Skills from Job Description
text = """[Your JD text here]"""  # Replace with actual JD text
JD_skills = extract_skills(text)

# 3. Extract Skills from Resume
resume_file = "/kaggle/input/resume/resume_v1.docx"
resume_text = extract_resume_text(resume_file)
resume_skills = extract_skills(resume_text)

# 4. Create Relationships and Update Knowledge Graph
# Get existing nodes
node_names = get_all_node_names()

# Generate relationships text
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0.9)
relationships = llm.invoke([
    ("system", "You are a knowledge graph relationship specialist. Create clear, ONLY really close relationships between technical concepts."),
    ("human", create_relationship_prompt(resume_skills, existing_knowledge=node_names))
])

# Transform and add new relationships
text = relationships.content
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(graph_documents)


relationships = llm.invoke([
    ("system", "You are a knowledge graph relationship specialist. Create clear, ONLY really close relationships between technical concepts."),
    ("human", create_relationship_prompt(JD_skills, existing_knowledge=node_names))
])

# Transform and add new relationships
text = relationships.content
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(graph_documents)

# 5. Calculate Match Score
test_single_user_and_jd(graph, JD_skills, resume_skills)